# Ictonyx Example: PyTorch Regression Variability Study

This notebook trains a small network on a synthetic regression problem and reports
the distribution of validation MSE across runs.

**Requirements:** `pip install ictonyx torch`

In [ ]:
import numpy as np
import torch
import torch.nn as nn

import ictonyx as ix
from ictonyx import (
    ModelConfig,
    PyTorchModelWrapper,
    ArraysDataHandler,
    run_variability_study,
)

print(f"Ictonyx v{ix.__version__}")
print(f"PyTorch v{torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 1. Generate Synthetic Data

We create a simple linear relationship with 5 features and Gaussian noise.
The true weights are known, so we can verify the model is learning something real.

In [ ]:
rng = np.random.RandomState(0)
X = rng.randn(500, 5).astype(np.float32)
true_weights = np.array([1.5, -2.0, 0.5, 0.0, 3.0], dtype=np.float32)
y = X @ true_weights + rng.randn(500).astype(np.float32) * 0.3

data_handler = ArraysDataHandler(X, y, test_size=0.2, val_size=0.2)

print(f"Samples: {len(X)}")
print(f"Features: {X.shape[1]}")
print(f"True weights: {true_weights}")
print(f"Noise level: 0.3")

## 2. Define the Model Builder

For regression, set `task='regression'` and use an appropriate loss (MSELoss).
The wrapper will track MSE instead of accuracy during training.

In [ ]:
def create_regressor(config: ModelConfig) -> PyTorchModelWrapper:
    model = nn.Sequential(
        nn.Linear(5, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
    )
    return PyTorchModelWrapper(
        model,
        criterion=nn.MSELoss(),
        optimizer_class=torch.optim.Adam,
        optimizer_params={'lr': config.get('learning_rate', 0.005)},
        task='regression',
    )

print(repr(create_regressor(ModelConfig())))

## 3. Run the Variability Study

10 runs, 50 epochs each. The question: how much does final MSE vary
across different random initializations?

In [ ]:
config = ModelConfig({
    'epochs': 50,
    'batch_size': 32,
    'learning_rate': 0.005,
    'verbose': 0,
})

results = run_variability_study(
    model_builder=create_regressor,
    data_handler=data_handler,
    model_config=config,
    num_runs=10,
    seed=42,
)

## 4. Examine Results

In [ ]:
print(results.summarize())

In [ ]:
print("Available metrics:", results.get_available_metrics())
print()
val_mse = results.get_metric_values('val_mse')
print("Per-run val_mse:")
for i, mse in enumerate(val_mse, 1):
    print(f"  Run {i}: {mse:.4f}")

print(f"\nBest:  {min(val_mse):.4f}")
print(f"Worst: {max(val_mse):.4f}")
print(f"Range: {max(val_mse) - min(val_mse):.4f}")

In [ ]:
# Summary DataFrame
results.to_dataframe()